In [29]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

#import time
#import argparse
#import multiprocessing


from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
#from IPython.display import display

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.9.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.9.* or later!')

In [30]:
# This is needed to display the images.
%matplotlib inline

In [31]:
from object_detection.utils import label_map_util
from object_detection.utils import ops as utils_ops
from object_detection.utils import visualization_utils as vis_util

In [32]:
MODEL_NAME = 'inference_graph/ssdlite_mobilenet_v2_coco_33104'
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'
PATH_TO_LABELS = os.path.join('annotations', 'labelmap-all.pbtxt')

In [33]:
NUM_CLASSES = 5

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES)
category_index = label_map_util.create_category_index(categories)

In [34]:
def run_inference_for_single_image(image, graph):
    if 'detection_masks' in tensor_dict:
        # Processamento de uma única imagem
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # É necessário atualizar a imagem para converter a máscara das coordenadas da caixa em coordenadas da imagem e ajustar o tamanho da imagem.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
    image_tensor = tf.compat.v1.get_default_graph().get_tensor_by_name('image_tensor:0')
    #print(image_tensor.eval, "\n")

    # Run inference
    output_dict = sess.run(tensor_dict,
                            feed_dict={image_tensor: np.expand_dims(image, 0)})
    #print(output_dict, "\n")
    # all outputs are float32 numpy arrays, so convert types as appropriate
    output_dict['num_detections'] = int(output_dict['num_detections'][0])
    output_dict['detection_classes'] = output_dict[
        'detection_classes'][0].astype(np.uint8)
    output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
    output_dict['detection_scores'] = output_dict['detection_scores'][0]
    if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict

In [35]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.compat.v1.GraphDef()
  with tf.compat.v2.io.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [36]:
#import glob

import pandas as pd

#targetPattern = r"CSGO_images\test\*.jpg"
#arquivos = glob.glob(targetPattern)

pasta = r"CSGO_images//train//"
fileExt = r".jpg"
fileExt_1 = r".JPG"
arquivos  = [_ for _ in os.listdir(pasta) if _.endswith(fileExt) or _.endswith(fileExt_1)]
arquivos

['1outa10.jpg',
 '1outa101.jpg',
 '1outa102.jpg',
 '1outa103.jpg',
 '1outa104.jpg',
 '1outa105.jpg',
 '1outa106.jpg',
 '1outa107.jpg',
 '1outa108.jpg',
 '1outa109.jpg',
 '1outa11.jpg',
 '1outa110.jpg',
 '1outa111.jpg',
 '1outa113.jpg',
 '1outa114.jpg',
 '1outa115.jpg',
 '1outa116.jpg',
 '1outa117.jpg',
 '1outa118.jpg',
 '1outa119.jpg',
 '1outa12.jpg',
 '1outa120.jpg',
 '1outa121.jpg',
 '1outa122.jpg',
 '1outa123.jpg',
 '1outa124.jpg',
 '1outa125.jpg',
 '1outa126.jpg',
 '1outa127.jpg',
 '1outa129.jpg',
 '1outa13.jpg',
 '1outa130.jpg',
 '1outa131.jpg',
 '1outa132.jpg',
 '1outa133.jpg',
 '1outa134.jpg',
 '1outa135.jpg',
 '1outa136.jpg',
 '1outa137.jpg',
 '1outa138.jpg',
 '1outa139.jpg',
 '1outa14.jpg',
 '1outa140.jpg',
 '1outa141.jpg',
 '1outa142.jpg',
 '1outa143.jpg',
 '1outa144.jpg',
 '1outa145.jpg',
 '1outa146.jpg',
 '1outa147.jpg',
 '1outa148.jpg',
 '1outa149.jpg',
 '1outa15.jpg',
 '1outa150.jpg',
 '1outa151.jpg',
 '1outa152.jpg',
 '1outa154.jpg',
 '1outa155.jpg',
 '1outa156.jpg',
 '1

In [37]:
test_csv = pd.read_csv('train_labels_1.csv')
arr = np.array(test_csv)
arr

array([['1outa10.jpg', 2, 0.3645833333333329, 0.4833984375,
        0.7621527777777779, 0.59765625],
       ['1outa101.jpg', 2, 0.42361111111111105, 0.43359375,
        0.7795138888888891, 0.6162109375],
       ['1outa102.jpg', 2, 0.421875, 0.4326171875, 0.7829861111111109,
        0.60546875],
       ...,
       ['IMG_9176.JPG', 2, 0.571666666666667, 0.31625,
        0.8383333333333329, 0.50625],
       ['IMG_9177.JPG', 2, 0.556666666666667, 0.56625, 0.83, 0.745],
       ['IMG_9177.JPG', 1, 0.111666666666667, 0.52125,
        0.3566666666666671, 0.72625]], dtype=object)

In [38]:
import cv2
previsoes = []
l = 0

Verdadeiro_positivo = 0
Falso_positivo = 0
Falso_negativo = 0
Verdadeiro_negativo = 0

for w in arquivos:
    image_np = cv2.imread(pasta + w)
    print("step:", l)
    l = l + 1 
    try:
        with detection_graph.as_default():
                with tf.compat.v1.Session() as sess:
                    
                    # Get handles to input and output tensors
                    ops = tf.compat.v1.get_default_graph().get_operations()
                    all_tensor_names = {output.name for op in ops for output in op.outputs}
                    tensor_dict = {}
                    
                    for key in [
                      'num_detections', 'detection_boxes', 'detection_scores',
                      'detection_classes', 'detection_masks'
                    ]:
                        tensor_name = key + ':0'
                        if tensor_name in all_tensor_names:
                            tensor_dict[key] = tf.compat.v1.get_default_graph().get_tensor_by_name(
                          tensor_name)
                            #print(tensor_dict[key], "\n")

                    #ret, image_np = cap.read()
                    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
                    image_np_expanded = np.expand_dims(image_np, axis=0)
                    #print("image_np_expanded", "\n")
                    # Actual detection.
                    output_dict = run_inference_for_single_image(image_np, detection_graph)
                    #print(output_dict, "\n")

                    test_result = tf.greater(output_dict['detection_scores'], [0.50])
                    j = 0

                    linha = []
                    previsoes_2 = []
                    linha.append(w)
                    
                    
                    for i in test_result.eval():
                        if i == True:
                            linha = []
                            linha.append(w)
                            linha.append(output_dict['detection_classes'][j])
                            linha.append(output_dict['detection_boxes'][j][0])
                            linha.append(output_dict['detection_boxes'][j][1])
                            linha.append(output_dict['detection_boxes'][j][2])
                            linha.append(output_dict['detection_boxes'][j][3])
                            #print(linha_1, "\n")
                            
                            #linha.append(linha_1)
                            #linha.append(linha_2)

                            previsoes.append(linha)
                            previsoes_2.append(linha)
                            print(linha, "\n")
                            
                        j = j + 1
                        
                    index = np.argwhere(arr == w)
                    print("index")
                    print("1", index)
                    print(len(index))
                    print("arr1", arr[index[0][0]][1])

                    if len(previsoes_2) != 0:       
                        
                        matrix_resultado = []
                        q = []
                        n = 0
                        for i in index:
                            k = []
                            m = 0
                            for g in previsoes_2:
                                Erro_cord = []
                                #print("if2", arr[i[0]][1] == g[1], "identificação de classe", arr[i[0]][1], g[1])
                                if arr[i[0]][1] == g[1]:
                                    Erro_cord.append(arr[i[0]][2] - g[2])
                                    Erro_cord.append(arr[i[0]][3] - g[3])
                                    Erro_cord.append(arr[i[0]][4] - g[4])
                                    Erro_cord.append(arr[i[0]][5] - g[5])
                                    erro_1 = tf.greater(Erro_cord, [0.05])
                                    erro_2 = tf.less(Erro_cord, [-0.05])
                                    with tf.compat.v1.Session() as sess:
                                        if any(o == True for o in erro_1.eval()) or any(o == True for o in erro_2.eval()):
                                            #print("Coordenada errada")
                                            k.append(False)
                                            
                                        else:                           
                                            #print("acetou")
                                            k.append(True)
                                            
                                else:
                                    #print("Classe errada")
                                    k.append(False)
                                m = m + 1        
                            q.append(k)
                            print("k", k)
                                
                            n = n + 1
                        print("q", q)
                        print("m", m)
                        print("n", n)
                        
                        df = pd.DataFrame(q)
                        for f in range(0, m, 1):
                            print("F1", f)
                            c = np.all(df[f] == False)
                            print("Coluna", f, c)
                            if c:
                                Falso_positivo = Falso_positivo + 1
                        
                        for f in range(0, n, 1):
                            print("F2", f)
                            c = np.any(q[f])
                            print("Linha", f, c)
                            if c:
                                Verdadeiro_positivo = Verdadeiro_positivo + 1
                            else:
                                Falso_negativo = Falso_negativo + 1
                    
                    else:
                        if arr[index[0][0]][1] == 0:
                            Verdadeiro_negativo = Verdadeiro_negativo + 1
                    
                        else:                        
                            Falso_negativo = Falso_negativo + 1
                            #print("não encontrou nada", "Falso_negativo", Falso_negativo)
                    
                    print("Falso_negativo", Falso_negativo)
                    print("Verdadeiro_positivo", Verdadeiro_positivo)
                    print("Falso_positivo", Falso_positivo)
                    print("Verdadeiro_negativo", Verdadeiro_negativo)  
                    
                    # Visualization of the results of a detection.
                    vis_util.visualize_boxes_and_labels_on_image_array(
                        image_np,
                        output_dict['detection_boxes'],
                        output_dict['detection_classes'],
                        output_dict['detection_scores'],
                        category_index,
                        instance_masks=output_dict.get('detection_masks'),
                        use_normalized_coordinates=True,
                        line_thickness=3)
                    #cv2.imshow('object_detection', cv2.resize(image_np, (800, 600)))
                    #while True:
                        #if cv2.waitKey(25) & 0xFF == ord('q'):
                            #cv2.destroyAllWindows()
                            #break

    except Exception as e:
        print(e)
        #cap.release()

print("Falso_negativo", Falso_negativo)
print("Verdadeiro_positivo", Verdadeiro_positivo)
print("Falso_positivo", Falso_positivo)
print("Verdadeiro_negativo", Verdadeiro_negativo)  


step: 0
['1outa10.jpg', 2, 0.38294438, 0.4823949, 0.76748097, 0.5976836] 

index
1 [[0 0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 0
Verdadeiro_positivo 1
Falso_positivo 0
Verdadeiro_negativo 0
step: 1
['1outa101.jpg', 2, 0.41567776, 0.43448615, 0.7682315, 0.6176659] 

index
1 [[1 0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 0
Verdadeiro_positivo 2
Falso_positivo 0
Verdadeiro_negativo 0
step: 2
['1outa102.jpg', 2, 0.41457248, 0.43686086, 0.7711812, 0.6144908] 

index
1 [[2 0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 0
Verdadeiro_positivo 3
Falso_positivo 0
Verdadeiro_negativo 0
step: 3
['1outa103.jpg', 2, 0.41715825, 0.43867207, 0.7804941, 0.61019695] 

index
1 [[3 0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 0
Verdadeiro_positivo 4
Falso_positivo 0
Verdadeiro_negativo 0
step: 4
['1outa104.j

k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 2
Verdadeiro_positivo 33
Falso_positivo 0
Verdadeiro_negativo 0
step: 35
['1outa134.jpg', 2, 0.43053335, 0.38205647, 0.69602364, 0.592517] 

index
1 [[35  0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 2
Verdadeiro_positivo 34
Falso_positivo 0
Verdadeiro_negativo 0
step: 36
['1outa135.jpg', 2, 0.42798486, 0.37752578, 0.69115126, 0.5899764] 

index
1 [[36  0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 2
Verdadeiro_positivo 35
Falso_positivo 0
Verdadeiro_negativo 0
step: 37
['1outa136.jpg', 2, 0.4201614, 0.38986605, 0.70129526, 0.59488773] 

index
1 [[37  0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 2
Verdadeiro_positivo 36
Falso_positivo 0
Verdadeiro_negativo 0
step: 38
['1outa137.jpg', 2, 0.41993254, 0.3894858, 0.6968414, 0.59744865] 

index
1 [[38  0]]
1
arr1 2
k 

k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 2
Verdadeiro_positivo 66
Falso_positivo 0
Verdadeiro_negativo 0
step: 68
['1outa165.jpg', 2, 0.42433095, 0.42976996, 0.80509865, 0.5729451] 

index
1 [[68  0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 2
Verdadeiro_positivo 67
Falso_positivo 0
Verdadeiro_negativo 0
step: 69
['1outa166.jpg', 2, 0.4225162, 0.43636903, 0.8131995, 0.5689565] 

index
1 [[69  0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 2
Verdadeiro_positivo 68
Falso_positivo 0
Verdadeiro_negativo 0
step: 70
['1outa168.jpg', 2, 0.4160101, 0.45486277, 0.8114388, 0.55169255] 

index
1 [[70  0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 2
Verdadeiro_positivo 69
Falso_positivo 0
Verdadeiro_negativo 0
step: 71
['1outa169.jpg', 2, 0.42445114, 0.46346524, 0.8115177, 0.54119915] 

index
1 [[71  0]]
1
arr1 2
k [

k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 2
Verdadeiro_positivo 99
Falso_positivo 0
Verdadeiro_negativo 0
step: 101
['1outa199.jpg', 2, 0.41805705, 0.36997545, 0.7340946, 0.59318817] 

index
1 [[101   0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 2
Verdadeiro_positivo 100
Falso_positivo 0
Verdadeiro_negativo 0
step: 102
['1outa2.jpg', 2, 0.38661337, 0.4806674, 0.7601782, 0.59902126] 

index
1 [[102   0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 2
Verdadeiro_positivo 101
Falso_positivo 0
Verdadeiro_negativo 0
step: 103
['1outa200.jpg', 2, 0.49857682, 0.3719486, 0.7774263, 0.5795995] 

index
1 [[103   0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 2
Verdadeiro_positivo 102
Falso_positivo 0
Verdadeiro_negativo 0
step: 104
['1outa201.jpg', 2, 0.4984265, 0.37213984, 0.77642924, 0.5808474] 

index
1 [[104   0]]


['1outa23.jpg', 2, 0.39120716, 0.48177347, 0.80256206, 0.6002772] 

index
1 [[133   0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 2
Verdadeiro_positivo 132
Falso_positivo 1
Verdadeiro_negativo 0
step: 134
['1outa230.jpg', 2, 0.50266504, 0.42420503, 0.79041624, 0.5859562] 

index
1 [[134   0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 2
Verdadeiro_positivo 133
Falso_positivo 1
Verdadeiro_negativo 0
step: 135
['1outa231.jpg', 2, 0.50068146, 0.4247231, 0.794403, 0.5907528] 

index
1 [[135   0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 2
Verdadeiro_positivo 134
Falso_positivo 1
Verdadeiro_negativo 0
step: 136
['1outa232.jpg', 2, 0.50179416, 0.41986758, 0.80242187, 0.5945824] 

index
1 [[136   0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 2
Verdadeiro_positivo 135
Falso_positivo 1
Verdadeiro_negativo 

...

['IMG_9018.JPG', 5, 0.395481, 0.2624304, 0.76023674, 0.71440315] 

index
1 [[2982    0]]
1
arr1 5
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 349
Verdadeiro_positivo 2632
Falso_positivo 358
Verdadeiro_negativo 0
step: 2979
['IMG_9019.JPG', 5, 0.26699516, 0.3443852, 0.8939301, 0.62345684] 

index
1 [[2983    0]]
1
arr1 5
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 349
Verdadeiro_positivo 2633
Falso_positivo 358
Verdadeiro_negativo 0
step: 2980
['IMG_9021.JPG', 5, 0.41179633, 0.32299194, 0.7566954, 0.64101505] 

index
1 [[2984    0]]
1
arr1 5
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 349
Verdadeiro_positivo 2634
Falso_positivo 358
Verdadeiro_negativo 0
step: 2981
['IMG_9023.JPG', 4, 0.2750929, 0.37533724, 0.798068, 0.73097396] 

index
1 [[2985    0]]
1
arr1 4
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 349
Verdadeiro_positivo 2635
Falso_pos

['IMG_9053.JPG', 5, 0.19294822, 0.69921935, 0.5780126, 0.9383192] 

['IMG_9053.JPG', 3, 0.34555897, 0.28094384, 0.8509978, 0.68899417] 

['IMG_9053.JPG', 4, 0.45146, 0.10412762, 0.7033942, 0.27676547] 

index
1 [[3018    0]
 [3019    0]
 [3020    0]
 [3021    0]
 [3022    0]]
5
arr1 5
k [True, False, False]
k [False, True, False]
k [False, False, True]
k [False, False, False]
k [False, False, False]
q [[True, False, False], [False, True, False], [False, False, True], [False, False, False], [False, False, False]]
m 3
n 5
F1 0
Coluna 0 False
F1 1
Coluna 1 False
F1 2
Coluna 2 False
F2 0
Linha 0 True
F2 1
Linha 1 True
F2 2
Linha 2 True
F2 3
Linha 3 False
F2 4
Linha 4 False
Falso_negativo 353
Verdadeiro_positivo 2668
Falso_positivo 360
Verdadeiro_negativo 0
step: 3001
['IMG_9054.JPG', 5, 0.1668587, 0.68430877, 0.5708809, 0.91117704] 

['IMG_9054.JPG', 4, 0.4228903, 0.08406249, 0.667935, 0.25096783] 

['IMG_9054.JPG', 3, 0.25385585, 0.26153404, 0.83073854, 0.66794646] 

['IMG_9054.JPG', 1, 0

['IMG_9086.JPG', 5, 0.13418297, 0.3365146, 0.6291334, 0.60744554] 

index
1 [[3059    0]]
1
arr1 5
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 366
Verdadeiro_positivo 2692
Falso_positivo 367
Verdadeiro_negativo 0
step: 3022
['IMG_9087.JPG', 2, 0.25492093, 0.34466273, 0.6806792, 0.56547296] 

['IMG_9087.JPG', 5, 0.26106876, 0.34366783, 0.6852029, 0.5665608] 

index
1 [[3060    0]]
1
arr1 5
k [False, True]
q [[False, True]]
m 2
n 1
F1 0
Coluna 0 True
F1 1
Coluna 1 False
F2 0
Linha 0 True
Falso_negativo 366
Verdadeiro_positivo 2693
Falso_positivo 368
Verdadeiro_negativo 0
step: 3023
['IMG_9089.JPG', 5, 0.33832934, 0.28367054, 0.7423012, 0.6206644] 

['IMG_9089.JPG', 2, 0.33832934, 0.28367054, 0.7423012, 0.6206644] 

index
1 [[3061    0]]
1
arr1 5
k [True, False]
q [[True, False]]
m 2
n 1
F1 0
Coluna 0 False
F1 1
Coluna 1 True
F2 0
Linha 0 True
Falso_negativo 366
Verdadeiro_positivo 2694
Falso_positivo 369
Verdadeiro_negativo 0
step: 3024
['IMG_9090.JPG

['IMG_9113.JPG', 4, 0.5403183, 0.15627196, 0.72298557, 0.30169815] 

['IMG_9113.JPG', 2, 0.58459646, 0.5033216, 0.74551004, 0.5865118] 

['IMG_9113.JPG', 2, 0.54667586, 0.68566763, 0.79595226, 0.86439776] 

['IMG_9113.JPG', 5, 0.5508089, 0.68413407, 0.80131483, 0.86275345] 

index
1 [[3094    0]
 [3095    0]
 [3096    0]
 [3097    0]
 [3098    0]]
5
arr1 3
k [False, False, False, False]
k [True, False, False, False]
k [False, False, False, False]
k [False, False, False, False]
k [False, False, False, True]
q [[False, False, False, False], [True, False, False, False], [False, False, False, False], [False, False, False, False], [False, False, False, True]]
m 4
n 5
F1 0
Coluna 0 False
F1 1
Coluna 1 True
F1 2
Coluna 2 True
F1 3
Coluna 3 False
F2 0
Linha 0 False
F2 1
Linha 1 True
F2 2
Linha 2 False
F2 3
Linha 3 False
F2 4
Linha 4 True
Falso_negativo 382
Verdadeiro_positivo 2715
Falso_positivo 384
Verdadeiro_negativo 0
step: 3044
['IMG_9114.JPG', 4, 0.7467295, 0.31581914, 0.9939964, 0.503826

['IMG_9153.JPG', 4, 0.5166107, 0.16565701, 0.7783478, 0.3657466] 

['IMG_9153.JPG', 5, 0.6064214, 0.49958396, 0.9209277, 0.7568953] 

index
1 [[3135    0]
 [3136    0]
 [3137    0]
 [3138    0]
 [3139    0]]
5
arr1 3
k [False, False]
k [False, False]
k [False, False]
k [True, False]
k [False, True]
q [[False, False], [False, False], [False, False], [True, False], [False, True]]
m 2
n 5
F1 0
Coluna 0 False
F1 1
Coluna 1 False
F2 0
Linha 0 False
F2 1
Linha 1 False
F2 2
Linha 2 False
F2 3
Linha 3 True
F2 4
Linha 4 True
Falso_negativo 400
Verdadeiro_positivo 2738
Falso_positivo 392
Verdadeiro_negativo 0
step: 3065
['IMG_9154.JPG', 5, 0.5599755, 0.5063095, 0.93652403, 0.7520391] 

['IMG_9154.JPG', 4, 0.5278456, 0.13817254, 0.7882838, 0.34988445] 

['IMG_9154.JPG', 2, 0.73776484, 0.2885751, 0.8881806, 0.4159913] 

['IMG_9154.JPG', 1, 0.4047709, 0.34973952, 0.52802193, 0.4327282] 

index
1 [[3140    0]
 [3141    0]
 [3142    0]
 [3143    0]
 [3144    0]]
5
arr1 5
k [True, False, False, False]

In [39]:
presisao = Verdadeiro_positivo/(Verdadeiro_positivo+Falso_positivo)
presisao

0.8750792644261256

In [40]:
seletividade = Verdadeiro_positivo/(Verdadeiro_positivo+Falso_negativo)
seletividade

0.8695652173913043

In [41]:
especificidade = Verdadeiro_negativo/(Verdadeiro_negativo+Falso_positivo)
especificidade

0.0

In [42]:
acuracidade = (Verdadeiro_positivo+Verdadeiro_negativo)/(Verdadeiro_positivo+Verdadeiro_negativo+Falso_negativo+Falso_positivo)
acuracidade

0.773542600896861